# Mushroom Classification with Deep Learning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/FractalN/blob/main/Mushroom_Classifier.ipynb)


In [24]:
# Check if we're running in Colab
import os
from pathlib import Path
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    # Clone repository (includes dataset in data/ folder)
    !git clone https://github.com/YOUR_USERNAME/FractalN.git
    %cd FractalN
    
    # Install additional requirements
    !pip install -r requirements.txt
    
    # Verify dataset is present
    !ls -R data/
    
    # Mount Google Drive for saving results
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Create results directory in Drive
    !mkdir -p "/content/drive/MyDrive/FractalN_Results"
else:
    # Verify dataset is present in local environment
    if not os.path.exists('data'):
        raise FileNotFoundError("Dataset not found! Please ensure 'data' directory exists.")
    print("Dataset found in local environment.")

Dataset found in local environment.


In [25]:
# Setup environment
import os
import sys
from src.gpu_config import setup_gpu
import tensorflow as tf

# Setup GPU
setup_gpu()

print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))

# Set results directory based on environment
RESULTS_DIR = '/content/drive/MyDrive/FractalN_Results' if IN_COLAB else 'results'

Detected CUDA version: 12.6

NVIDIA GPU Information:
Thu Dec 26 07:30:23 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77                 Driver Version: 565.77         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650        On  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P0             14W /   50W |     369MiB /   4096MiB |      3%      Default |
|                                         |                        |                  N

In [26]:
# Setup paths and check for cached processed data
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Define paths
    DRIVE_ROOT = '/content/drive/MyDrive/FractalN_Data'
    AUGMENTED_DATA_PATH = f"{DRIVE_ROOT}/final_processed_data.zip"  # Final processed data after all steps
    RESULTS_DIR = f"{DRIVE_ROOT}/Results"
    
    # Create directories
    !mkdir -p "{DRIVE_ROOT}"
    !mkdir -p "{RESULTS_DIR}"
    
    # Check if final processed data exists in Drive
    NEED_PROCESSING = not os.path.exists(AUGMENTED_DATA_PATH)
else:
    RESULTS_DIR = 'results'
    NEED_PROCESSING = not os.path.exists('data/processed')

## 1. Complete Data Pipeline
Process data in three steps:
1. Organize raw data
2. Augment organized data
3. Preprocess augmented data

In [27]:
if NEED_PROCESSING:
    print("No cached processed data found. Starting complete data pipeline...")
    
    # Step 1: Organize raw data
    print("\n1. Organizing raw data...")
    from utils.reorganize_data import reorganize_mushroom_data
    reorganize_mushroom_data()
    
    # Step 2: Augment organized data
    print("\n2. Augmenting organized data...")
    from utils.augment_mushroom_data import augment_mushroom_data
    augment_mushroom_data(target_count=20000)
    
    # Step 3: Preprocess augmented data
    print("\n3. Preprocessing augmented data...")
    from utils.preprocess_data import preprocess_dataset
    preprocess_dataset(
        data_dir='data/mushroom_data',
        output_dir='data/processed',
        test_size=0.2,
        img_size=(224, 224)
    )
    
    # Cache the final processed data
    if IN_COLAB:
        print("\nSaving final processed data to Google Drive...")
        !zip -r "{AUGMENTED_DATA_PATH}" data/processed
        print(f"Final processed data saved to: {AUGMENTED_DATA_PATH}")
else:
    print("Found cached processed data. Loading...")
    if IN_COLAB:
        !unzip -q "{AUGMENTED_DATA_PATH}" -d "data/"
    print("Cached data loaded successfully!")

# Verify final data structure and distribution
!tree data/processed -L 3

print("\nFinal dataset distribution:")
for split in ['train', 'test', 'val']:
    print(f"\n{split.capitalize()} set:")
    for category in ['poisonous', 'edible']:
        count = len(list(Path(f'data/processed/{split}/{category}').glob('*.jpg')))
        print(f"{category}: {count} images")

Found cached processed data. Loading...
Cached data loaded successfully!
/usr/bin/bash: line 1: tree: command not found

Final dataset distribution:

Train set:
poisonous: 1627 images
edible: 1864 images

Test set:
poisonous: 338 images
edible: 387 images

Val set:
poisonous: 288 images
edible: 329 images


## 2. Model Training
Train using the fully processed dataset

In [ ]:
import os
from pathlib import Path
def verify_and_setup():
    """Verify data structure and setup for training"""
    if not os.path.exists('data/processed'):
        print("No processed data found. Will run preprocessing.")
        return True
    
    train_dir = Path('data/processed/train')
    if not train_dir.exists():
        print("No training directory found. Will run preprocessing.")
        return True
        
    edible_count = len(list((train_dir / 'edible').glob('*.[Jj][Pp][Gg]')))
    poisonous_count = len(list((train_dir / 'poisonous').glob('*.[Jj][Pp][Gg]')))
    
    if edible_count == 0 or poisonous_count == 0:
        print("Missing data in training directories. Will run preprocessing.")
        return True
        
    print(f"Found existing dataset: {edible_count} edible, {poisonous_count} poisonous")
    return False

# Setup Python path for imports
import os
import sys
notebook_dir = os.path.abspath('')
src_dir = os.path.join(notebook_dir, 'src')
if src_dir not in sys.path:
    sys.path.append(notebook_dir)
    print(f"Added {notebook_dir} to Python path")

from src.train import train_model
from src.model import create_model
need_preprocess = verify_and_setup()
model, history = train_model(preprocess=need_preprocess)

print("\nDisplaying training results...")
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread('training_history.png')
plt.figure(figsize=(15, 5))
plt.imshow(img)
plt.axis('off')
plt.show()

print("\nTraining Metrics:")
with open('training_metrics.txt', 'r') as f:
    print(f.read())

Added /mnt/external/Development/fractalN to Python path
Detected CUDA version: 12.6

NVIDIA GPU Information:
Thu Dec 26 07:33:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77                 Driver Version: 565.77         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650        On  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P0             13W /   50W |     352MiB /   4096MiB |     30%      Default |
|                              

/mnt/external/Development/fractalN/.venv/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Found 617 files belonging to 2 classes.
Found 725 files belonging to 2 classes.

Class weights being used:
Class 0: 0.9364
Class 1: 1.0728
Epoch 1/150


I0000 00:00:1735177747.824846 1311071 cuda_dnn.cc:529] Loaded cuDNN version 90501


Error during training: Exception encountered when calling Dropout.call().

{{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 

Arguments received by Dropout.call():
  • inputs=tf.Tensor(shape=(8, 37, 37, 256), dtype=float16)
  • training=True

Please ensure you have:
1. Uploaded your data correctly
2. Run with preprocess=True for first time setup
3. Have the correct directory structure


ResourceExhaustedError: Exception encountered when calling Dropout.call().

[1m{{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: [0m

Arguments received by Dropout.call():
  • inputs=tf.Tensor(shape=(8, 37, 37, 256), dtype=float16)
  • training=True

: 

## 4. Model Evaluation and Prediction
Test the trained model on sample images

In [ ]:
from src.predict import predict_mushroom
import random
from pathlib import Path

def test_random_images(num_samples=3):
    test_dir = Path('data/processed/test')
    
    for category in ['edible', 'poisonous']:
        print(f"\nTesting {category} mushrooms:")
        category_path = test_dir / category
        image_files = list(category_path.glob('*.[Jj][Pp][Gg]'))
        
        for _ in range(num_samples):
            test_image = random.choice(image_files)
            print(f"\nImage: {test_image.name}")
            prediction, confidence = predict_mushroom(
                'mushroom_classifier.keras',
                str(test_image)
            )
            print(f"Predicted: {prediction}")
            print(f"Confidence: {confidence:.2%}")

# Test model on random images
test_random_images()

## 5. Save Model and Results

In [ ]:
# Save results to Google Drive if in Colab
if IN_COLAB:
    !cp mushroom_classifier.keras "{RESULTS_DIR}/"
    !cp best_model.keras "{RESULTS_DIR}/"
    !cp training_history.png "{RESULTS_DIR}/"
    !cp training_metrics.txt "{RESULTS_DIR}/"
    !cp training_log.csv "{RESULTS_DIR}/"
    print(f"Model and results saved to Google Drive: {RESULTS_DIR}")
else:
    !mkdir -p results
    !cp mushroom_classifier.keras results/
    !cp best_model.keras results/
    !cp training_history.png results/
    !cp training_metrics.txt results/
    !cp training_log.csv results/
    print("Model and results saved in 'results' directory")